## Loading The Dataset And the Libraries

In [1]:
import pandas as pd 

In [2]:
df = pd.read_csv(r'F:\Lerning\Case Studies\Retail Sales\retail_sales.csv')

In [3]:
df.head()

,Order_ID,Order_Date,Customer_ID,Customer_Name,Region,Product_Category,Product_Name,Quantity,Unit_Price,Discount,Total_Sales,Payment_Method,Delivery_Status
0,O-1000,2024-01-01,C-208,Jane Smith,South,Furniture,T-Shirt,10,805.52,0.01,7974.6480,Credit Card,Pending
1,O-1001,2024-01-02,C-365,Ali Hassan,North,Furnitre,Sofa,3,1550.01,0.25,3487.5225,Cash,Cancelled
2,O-1002,2024-01-03,C-498,Jane Smith,West,Furniture,Laptop,1,1187.38,0.10,1068.6420,Bank Transfer,Pending
3,O-1003,2024-01-04,C-371,Ali Hassan,East,Electronics,Headphones,2,932.39,0.07,1734.2454,Cash,Cancelled
4,O-1004,2024-01-05,C-218,Jane Smith,West,Clothing,Sofa,10,279.37,0.17,2318.7710,Cash,Returned


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 510 entries, 0 to 509
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Order_ID          510 non-null    object 
 1   Order_Date        510 non-null    object 
 2   Customer_ID       510 non-null    object 
 3   Customer_Name     475 non-null    object 
 4   Region            510 non-null    object 
 5   Product_Category  483 non-null    object 
 6   Product_Name      510 non-null    object 
 7   Quantity          510 non-null    int64  
 8   Unit_Price        486 non-null    object 
 9   Discount          510 non-null    float64
 10  Total_Sales       510 non-null    float64
 11  Payment_Method    510 non-null    object 
 12  Delivery_Status   510 non-null    object 
dtypes: float64(2), int64(1), object(10)
memory usage: 51.9+ KB


## Cleaning

Remove Duplicate

In [5]:
df = df.drop_duplicates()

Handle Missing Values

In [6]:
df["Customer_Name"] = df["Customer_Name"].fillna("Unknown")

df["Product_Category"] = df["Product_Category"].replace(
    {
        "Electrnics": "Electronics"
        , "Furnture": "Furniture"
}).fillna("Other")
df = df.dropna(subset = ["Unit_Price"])

Remove currency sympols, convert to fload (Unit_Price)

In [7]:
df["Unit_Price"] = df["Unit_Price"].astype(str).str.replace("[$£]", "", regex = True )
df['Unit_Price'] = df['Unit_Price'].astype(float)

Fix Dates (YYYY-MM-DD)

In [8]:
df['Order_Date'] = pd.to_datetime(df['Order_Date'], errors = 'coerce').dt.strftime('%Y-%m-%d')

Handle Negative Sales

In [9]:
df["Total_Sales"] = df["Quantity"] * df["Unit_Price"] * (1 - df["Discount"])

Recalculate Total_Sales

In [10]:
df["Total_Sales"] = df["Total_Sales"] * df["Unit_Price"] * (1 - df["Discount"])

Outlier: cap using IQR

In [12]:
for col in ["Total_Sales", "Discount"]:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower, upper = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
    df[col] = df[col].clip(lower, upper)

Feature Engineering

In [13]:
df["Order_Year"] = pd.to_datetime(df["Order_Date"]).dt.year
df["Order_Month"] = pd.to_datetime(df["Order_Date"]).dt.month

Recalculate Total_Sales

In [14]:
df["Total_Sales"] = df["Quantity"] * df["Unit_Price"] * (1 - df["Discount"])

Save Cleaned Data

In [17]:
df.to_csv(r'F:\Lerning\Case Studies\Retail Sales\1_Data Preparation\retail_sales_cleaned_python.csv', index = False)